# Data Science & LLM Technical Assessment Part 1

## Load dataframe and view shape, data types and missing values

In [ ]:
import pandas as pd
path = './data.csv'
df = pd.read_csv(path)

#print dataframe shape
print("Dataframe shape:", df.shape)

#show dataframe
df.head(2)

In [ ]:
#show data types in dataframe
df.dtypes

In [ ]:
#show missing values
df.isna().sum()

## Produce ydata profile report

In [ ]:
# #load profile report
# from ydata_profiling import ProfileReport

# #create profile report
# profile = ProfileReport(df, title="Profiling Report")

# #save to local filepath
# path = './ydata_report.html'
# profile.to_file(path)

## Feature distributions

### readmitted_30_days (outcome variable)

In [ ]:
import plotly.express as px

#create value count DataFrame
tmp_df = df['readmitted_30_days'].value_counts().reset_index()
tmp_df.columns = ['readmission_status', 'count']

#create h-barplot
fig = px.bar(
    tmp_df,
    x='count',
    y='readmission_status',
    orientation='h',
    text='count',
    title='Readmission Within 30 Days'
)

#format
fig.update_layout(
    xaxis_title='Number of Patients',
    yaxis_title='Readmission Status',
    yaxis=dict(type='category'),  # ensures discrete axis
    showlegend=False,
    template='plotly_white',
    bargap=0.3
)

#update formatting
fig.update_traces(textposition='outside')

#show plot
fig.show()

- 2:1 non-readmitted to readmitted
- moderately imbalanced across outcome variable
- explore synthetic upsampling to improve peformance (low priority)

### patient_id

In [ ]:
#check patient id is unique
assert len(df.patient_id.unique()) == df.shape[0], "patient_id is not unique check variable!"

- patient_id is unique

### gender 

In [ ]:
#create value count DataFrame
tmp_df = df['gender'].value_counts().reset_index()
tmp_df.columns = ['gender', 'count']

#create h-barplot
fig = px.bar(
    tmp_df,
    x='count',
    y='gender',
    orientation='h',
    text='count',
    title='Gender'
)

#format
fig.update_layout(
    xaxis_title='Number of Patients',
    yaxis_title='Gender',
    yaxis=dict(type='category'),  # ensures discrete axis
    showlegend=False,
    template='plotly_white',
    bargap=0.3
)

#update formatting
fig.update_traces(textposition='outside')

#show plot
fig.show()

- well-balanced feature, no underepresented group

### diagnosis_code

In [ ]:
#create value count DataFrame
tmp_df = df['diagnosis_code'].value_counts().reset_index()
tmp_df.columns = ['diagnosis_code', 'count']

#create h-barplot
fig = px.bar(
    tmp_df,
    x='count',
    y='diagnosis_code',
    orientation='h',
    text='count',
    title='Diagnosis Code'
)

#format
fig.update_layout(
    xaxis_title='Number of Patients',
    yaxis_title='Diagnosis Code',
    yaxis=dict(type='category'),  # ensures discrete axis
    showlegend=False,
    template='plotly_white',
    bargap=0.3
)

#update formatting
fig.update_traces(textposition='outside')

#show plot
fig.show()

- 4 unique values
- balanced disbution across D001, D002 and D004
- DOO3 moderately underepresented compared to mean

### num_previous_admissions

In [ ]:
#create boxplot
fig = px.box(
    df,
    y='num_previous_admissions',
    points='all',  # Show individual data points
    title='Distribution of Previous Admissions',
    boxmode='overlay'
)

#specify height
height, width = 600, 600

#formatting
fig.update_layout(
    yaxis_title='Number of Previous Admissions',
    template='plotly_white',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

- 6 unique values
- median value of 1 previous admission
- RHS skew to distribution, both Q1 and Q2 are 1, Q3 is 3

### medication_type

In [ ]:
#create value count DataFrame
tmp_df = df['medication_type'].value_counts().reset_index()
tmp_df.columns = ['medication_type', 'count']

#create h-barplot
fig = px.bar(
    tmp_df,
    x='count',
    y='medication_type',
    orientation='h',
    text='count',
    title='Medication Type'
)

#format
fig.update_layout(
    xaxis_title='Number of Patients',
    yaxis_title='Medication Type',
    yaxis=dict(type='category'),  # ensures discrete axis
    showlegend=False,
    template='plotly_white',
    bargap=0.3
)

#update formatting
fig.update_traces(textposition='outside')

#show plot
fig.show()

- 3 unique medication types
- well balanced representation across each unique medication type

### length_of_stay

In [ ]:
#create boxplot
fig = px.box(
    df,
    y='length_of_stay',
    points='all',  # Show individual data points
    title='Distribution of Previous Admissions',
    boxmode='overlay'
)

#specify height
height, width = 600, 600

#formatting
fig.update_layout(
    yaxis_title='Length of Stay',
    template='plotly_white',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

In [ ]:
#create histogram
fig = px.histogram(
    df,
    x='length_of_stay',
    title='Length of Stay',
    nbins=14 #ostensibly a barplot as nbins == length_of_stay.unique
    )

#specify height
height, width = 600, 900

#formatting
fig.update_layout(
    xaxis_title='Length of Stay',
    yaxis_title='Number of Patients',
    template='plotly_white',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

- 14 unique values
- median stay of 8 days
- multimodal distribution with small peaks at 2, 7-8 and 11
- number of datapoints per unique value is small

### age

In [ ]:
#create histogram
fig = px.histogram(
    df,
    x='age',
    title='Age',
    nbins=20 #ostensibly a barplot as nbins == length_of_stay.unique
    )

#specify height
height, width = 600, 900

#formatting
fig.update_layout(
    xaxis_title='Age',
    yaxis_title='Number of Patients',
    template='plotly_white',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

- ages of patients in the sample population varied between 20 - 89
- median age of 54 years old
- multimodal distribution with a large peak between 20 - 24 years and mild peak between 70 - 84 years

## Joint feature distributions across outcome variable (Readmission)

### gender

#### Count plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'gender'

#create grouped count DataFrame
grouped_df = df.groupby([independent_var, dependent_var]).size().reset_index(name='count')

fig = px.bar(grouped_df, 
             x=dependent_var,
             y='count',
             color=independent_var, 
             barmode='group',
             title='Readmission vs Gender')

#specify height
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Number of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show fig
fig.show()

#### Proportion plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'gender'

#group by dependent and independent vars and count occurrences
grouped_df = df.groupby([dependent_var, independent_var]).size().reset_index(name='count')

#get total counts for each readmission group
total_counts = grouped_df.groupby(dependent_var)['count'].sum().reset_index(name='group_total')

#merge group totals back to grouped_df
grouped_df = grouped_df.merge(total_counts, on=dependent_var)

#calculate proportion within each readmission group
grouped_df['proportion'] = grouped_df['count'] / grouped_df['group_total']

#create bar plot using proportions
fig = px.bar(
    grouped_df,
    x=dependent_var,
    y='proportion',
    color=independent_var,
    barmode='group',
    title='Proportion of Gender within Each Readmission Group'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Proportion of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show plot
fig.show()

#### Observations

- women contributed a smaller overall proportion of the patients readmitted within 30 days than those who were not readmitted within 30 days (54.8% vs 50.8% to 1.dp)

### diagnosis_code

#### Count plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'diagnosis_code'

#create a grouped count DataFrame
grouped_df = df.groupby([dependent_var, independent_var]).size().reset_index(name='count')

fig = px.bar(grouped_df, 
             x=dependent_var,
             y='count',
             color=independent_var, 
             barmode='group',
             title='Readmission vs Diagnosis Code')

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Number of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show plot
fig.show()

#### Proportion plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'diagnosis_code'

#group by dependent and independent vars and count occurrences
grouped_df = df.groupby([dependent_var, independent_var]).size().reset_index(name='count')

#get total counts for each readmission group
total_counts = grouped_df.groupby(dependent_var)['count'].sum().reset_index(name='group_total')

#merge group totals back to grouped_df
grouped_df = grouped_df.merge(total_counts, on=dependent_var)

#calculate proportion within each readmission group
grouped_df['proportion'] = grouped_df['count'] / grouped_df['group_total']

#create bar plot using proportions
fig = px.bar(
    grouped_df,
    x=dependent_var,
    y='proportion',
    color=independent_var,
    barmode='group',
    title='Proportion of Diagnosis Codes within Each Readmission Group'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Proportion of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show plot
fig.show()

#### Obvservations

- D001 had similar representation across readmitted and non-readmitted individuals
- D002 had a smaller representation in readmitted individuals
- D003 had a slightly larger representation in readmitted individuals 
- D004 had similar representation across readmitted and non-readmitted individuals

### num_previous_admissions

#### Proportion plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'num_previous_admissions'

#group by dependent and independent vars and count occurrences
grouped_df = df.groupby([dependent_var, independent_var]).size().reset_index(name='count')

#get total counts for each readmission group
total_counts = grouped_df.groupby(dependent_var)['count'].sum().reset_index(name='group_total')

#merge group totals back to grouped_df
grouped_df = grouped_df.merge(total_counts, on=dependent_var)

#calculate proportion within each readmission group
grouped_df['proportion'] = grouped_df['count'] / grouped_df['group_total']

#create bar plot using proportions
fig = px.bar(
    grouped_df,
    x=dependent_var,
    y='proportion',
    color=independent_var,
    barmode='group',
    title='Proportion of Number of Previous Admissions within Each Readmission Group'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Proportion of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show plot
fig.show()

#### Box plot

In [ ]:
#declare variables
independent_var = 'num_previous_admissions'
dependent_var = 'readmitted_30_days'         

#create boxplot
fig = px.box(
    df,
    x=dependent_var,
    y=independent_var,
    color=dependent_var,
    title='Readmission vs Number of Previous Admissions',
    template='plotly_white'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Number of Previous Admissions',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

#### Density plot

In [ ]:
#declare variables
independent_var = 'num_previous_admissions'
dependent_var = 'readmitted_30_days'

#create violin plot
fig = px.violin(
    df,
    y=independent_var,
    x=dependent_var,
    color=dependent_var,
    box=False,            # Remove boxplot elements
    points=False,         # Remove individual data points
    hover_data=df.columns,
    title='Readmission vs Number of Previous Admissions',
    template='plotly_white'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Number of Previous Admissions',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

#### Obvservations

- individuals with one previous admission had a moderately higher density in individuals who went on to be readmitted within 30 days than those who did not go on to be readmitted
- individuals with 2 previous admissions had a moderately lower density in individuals who went on to be readmitted within 30 days than those who did not go on to be readmitted
- individuals with 5 previous admissions had a moderately lower density in individuals who went on to be readmitted within 30 days than those who did not go on to be readmitted, although the population size was small and the impact of variance could be exaggerated
- individuals with 6 previous admissions had a moderately higher density in individuals who went on to be readmitted within 30 days than those who did not go on to be readmitted, although the population size was small and the impact of variance could be exaggerated

### medication_type

#### Count plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'medication_type'

#create a grouped count DataFrame
grouped_df = df.groupby([dependent_var, independent_var]).size().reset_index(name='count')

fig = px.bar(grouped_df, 
             x=dependent_var,
             y='count',
             color=independent_var, 
             barmode='group',
             title='Readmission vs Medication Type')

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Number of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show plot
fig.show()

#### Proportion plot

In [ ]:
#declare variables
dependent_var = 'readmitted_30_days'
independent_var = 'medication_type'

#group by dependent and independent vars and count occurrences
grouped_df = df.groupby([dependent_var, independent_var]).size().reset_index(name='count')

#get total counts for each readmission group
total_counts = grouped_df.groupby(dependent_var)['count'].sum().reset_index(name='group_total')

#merge group totals back to grouped_df
grouped_df = grouped_df.merge(total_counts, on=dependent_var)

#calculate proportion within each readmission group
grouped_df['proportion'] = grouped_df['count'] / grouped_df['group_total']

#create bar plot using proportions
fig = px.bar(
    grouped_df,
    x=dependent_var,
    y='proportion',
    color=independent_var,
    barmode='group',
    title='Proportion of Medication Types within Each Readmission Group'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Proportion of Patients',
    template='plotly_white',
    showlegend=True,
    height=height,
    width=width
)

#show plot
fig.show()

#### Observations

- individuals on Type A medication had a mildly lower proportional representation in individuals who went on to be readmitted vs those who did not
- individuals on Type C medication had a mildly higher proportional representation in individuals who went on to be readmitted vs those who did not

### length_of_stay

#### Box plot

In [ ]:
#declare variables
independent_var = 'length_of_stay'
dependent_var = 'readmitted_30_days'       

#create boxplot
fig = px.box(
    df,
    x=dependent_var,
    y=independent_var,
    color=dependent_var,
    title='Readmission vs Number of Length of Stay',
    template='plotly_white'
)

#specify height and width
height, width = 600, 900

#formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Length of Stay',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

#### Density plot

In [ ]:
#declare variables
independent_var = 'length_of_stay'
dependent_var = 'readmitted_30_days'  # binary

#create violin plot
fig = px.violin(
    df,
    y=independent_var,
    x=dependent_var,
    color=dependent_var,
    box=False,            # Remove boxplot elements
    points=False,         # Remove individual data points
    hover_data=df.columns,
    title='Readmission vs Number of Length of Stay',
    template='plotly_white'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Length of Stay',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

#### Observations

- length_of_stay values ranged between 1 - 14 days
- median length of stay between readmitted and non-readmitted patients 8 
- IQR for patients that were not readmitted was between 4 - 11 days
- IQR for patients that were readmitted was shifted up by a day with a Q1 value of 5 and Q3 value of 
- Patients that stayed between 10 - 14 days were had a higher density in readmitted patients compared the non-readmitted equivalent
- Patients that stayed between 1 - 4 days had a lower density in readmitted patients compared to the non-readmitted equivalent

### age

#### Boxplot

In [ ]:
#declare variables
independent_var = 'age'
dependent_var = 'readmitted_30_days'

#create boxplot
fig = px.box(
    df,
    x=dependent_var,
    y=independent_var,
    color=dependent_var,
    title='Readmission vs Age',
    template='plotly_white'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Age',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

#### Density plot

In [ ]:
#declare variables
independent_var = 'age'
dependent_var = 'readmitted_30_days'  # binary

#create violin plot
fig = px.violin(
    df,
    y=independent_var,
    x=dependent_var,
    color=dependent_var,
    box=False,            # Remove boxplot elements
    points=False,         # Remove individual data points
    hover_data=df.columns,
    title='Age Distribution by Readmission',
    template='plotly_white'
)

#specify height and width
height, width = 600, 900

#update formatting
fig.update_layout(
    xaxis_title='Readmission within 30 Days',
    yaxis_title='Age',
    showlegend=False,
    height=height,
    width=width
)

#show plot
fig.show()

#### Observations

- same total range between individuals who were readmitted and those who were not (20 - 89 years)
- median age of readmitted individuals was higher that the equivalent in patients who were not readmitted, 58 year and 54 years in the respective order
- readmitted individuals were more evenly spread across a wider range of ages, shown by the larger interquartile range, 27 - 73 years for readmitted inviduals vs 38 - 73.5 years for those who were not readmitted
- higher density in readmitted individuals between 20 - 30 years and between 65 - 75 years
- lower density in readmitted individuals between 34 - 62 years

## Feature engineering

### num_previous_admissions - encode

In [ ]:
#ignore values 5 & 6 due to low sample size

#create columns for 1 and 2 previous admissions
df['e_num_previous_admissions_1'] = 0
df['e_num_previous_admissions_2'] = 0 

#assign 1 where num_previous_admissions == 1
df.loc[df['num_previous_admissions'] == 1, 'e_num_previous_admissions_1'] = 1
df.loc[df['num_previous_admissions'] == 2, 'e_num_previous_admissions_2'] = 1

#show df
df.head(2)

### age - encode

In [ ]:
#create binary indicator columns (already initialized to 0)
df['e_age_20to30'] = 0
df['e_age_65to75'] = 0
df['e_age_34to62'] = 0

# set to 1 where the condition is met
df.loc[(df['age'] >= 20) & (df['age'] <= 30), 'e_age_20to30'] = 1
df.loc[(df['age'] >= 65) & (df['age'] <= 75), 'e_age_65to75'] = 1
df.loc[(df['age'] >= 34) & (df['age'] <= 62), 'e_age_34to62'] = 1

#show df
df.head(2)

### length_of_stay - encode

In [ ]:
#create binary indicator columns (already initialized to 0)
df['e_length_of_stay_1to4'] = 0
df['e_length_of_stay_10to14'] = 0

#set to 1 where the condition is met
df.loc[(df['length_of_stay'] >= 1) & (df['length_of_stay'] <= 4), 'e_length_of_stay_1to4'] = 1
df.loc[(df['length_of_stay'] >= 10) & (df['length_of_stay'] <= 14), 'e_length_of_stay_10to14'] = 1

#show df
df.head(2)

### medication_type - encode

In [ ]:
#create binary indicator columns for medication_type C
df['e_medication_type_c'] = 0

#set to 1 where the condition is met
df.loc[df['medication_type'] == 'Type C', 'e_medication_type_c'] = 1

#show df
df.head(2)

### diagnosis_code - encode

In [ ]:
#create binary indicator columns for diagnosis_code d003
df['e_diagnosis_code_d003'] = 0

#set to 1 where the condition is met
df.loc[df['diagnosis_code'] == 'D003', 'e_diagnosis_code_d003'] = 1

#show df
df.head(2)

## Benchmark modelling

### Re-ordering dataframe

In [ ]:
#create new order for columns
ordered_columns = [
    'patient_id',
    'age',
    'gender',
    'diagnosis_code',
    'num_previous_admissions',
    'medication_type',
    'length_of_stay',
    'discharge_note',
    'e_num_previous_admissions_1',
    'e_num_previous_admissions_2',
    'e_age_20to30',
    'e_age_65to75',
    'e_age_34to62',
    'e_length_of_stay_1to4',
    'e_length_of_stay_10to14',
    'e_medication_type_c',
    'e_diagnosis_code_d003',
    'readmitted_30_days'
]

#list engineered features
engineered_features = [
    'e_num_previous_admissions_1',
    'e_num_previous_admissions_2',
    'e_age_20to30',
    'e_age_65to75',
    'e_age_34to62',
    'e_length_of_stay_1to4',
    'e_length_of_stay_10to14',
    'e_medication_type_c',
    'e_diagnosis_code_d003',
]

#re-order df
df = df[ordered_columns]

#show new df
df.head(2)

### Creating a dataframe for benchmarking

In [ ]:
#create list of benchmark columns
benchmark_cols = [

    #raw input features - discharge note removed
    'age',
    'gender',
    'diagnosis_code',
    'num_previous_admissions',
    'medication_type',
    'length_of_stay',

    #target variable
    'readmitted_30_days'
]

#create benchmark_df
benchmark_df = df[benchmark_cols].copy()

#show benchmark_df
benchmark_df.head(2)

### Random Forest

#### Pre-processing

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#columns to encode
string_cols = ['gender', 'diagnosis_code', 'medication_type']

#instantiate OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# ft and transform on string columns
encoded_array = ohe.fit_transform(benchmark_df[string_cols])

#create a DataFrame from the encoded array with meaningful column names
encoded_df = pd.DataFrame(
    encoded_array,
    columns=ohe.get_feature_names_out(string_cols),
    index=benchmark_df.index
)

#drop original string columns and concatenate encoded columns
ranf_benchmark_df = pd.concat([
    benchmark_df.drop(columns=string_cols),
    encoded_df
], axis=1)

#show df
ranf_benchmark_df.head(2)

#### Dataset creation

In [ ]:
from sklearn.model_selection import train_test_split

#create X and y dataframe slices
X = ranf_benchmark_df.drop(columns='readmitted_30_days').copy()
y = ranf_benchmark_df[['readmitted_30_days']].copy()

#create stratified train-test split to handle class imbalance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

#show shapes of dataframes
print("\nDataset Shapes:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

#show class distribution in y_train and y_test
print("\nClass distributions:")
print(y_train['readmitted_30_days'].value_counts(normalize=True))
print(y_test['readmitted_30_days'].value_counts(normalize=True))

#### Model training

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#instantiate Random Forest model using out-of-box hyperparameters
rf_model = RandomForestClassifier(
    n_estimators=100,               
    criterion='gini',               
    max_depth=None,                
    min_samples_split=2,          
    min_samples_leaf=1,            
    min_weight_fraction_leaf=0.0,  
    max_features='sqrt',          
    max_leaf_nodes=None,           
    min_impurity_decrease=0.0,     
    bootstrap=True,                
    oob_score=False,               
    n_jobs=None,                   
    random_state=42,               
    verbose=0,                     
    warm_start=False,              
    class_weight=None,             
    ccp_alpha=0.0,                 
    max_samples=None        
)

# Fit the model
rf_model.fit(X_train, y_train.values.ravel())  # ravel to avoid warning for 2D y

#### Performance

In [ ]:
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, f1_score, fbeta_score, roc_auc_score, roc_curve
)

import matplotlib.pyplot as plt

%matplotlib inline

#predict class labels
y_pred = rf_model.predict(X_test)

#predict class probabilities for ROC AUC
y_probs = rf_model.predict_proba(X_test)[:, 1]

#accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

#F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.3f}")

#F2 Score
f2 = fbeta_score(y_test, y_pred, beta=2)
print(f"F2 Score: {f2:.3f}")

#ROC AUC Score
roc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {roc:.3f}")

#confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf_model.classes_)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

#### SHAP plot

In [ ]:
import shap
import matplotlib.pyplot as plt

#create SHAP TreeExplainer
explainer = shap.TreeExplainer(rf_model)

#compute SHAP values for the test data
shap_values = explainer(X_test)

#extract SHAP values for readmitted
shap_values_class1 = shap_values[..., 1]

#plot SHAP bar plot
plt.figure(figsize=(12, 6))
shap.plots.bar(shap_values_class1, max_display=10)

### Observations

- The benchmark model showed weak predictive ability across all metrics, falling below the baseline accuracy of 0.66 (accuracy achieved when only selecting the most prevalent class in the outcome variable) 
- The model also favoured the predicting the negative class with a ratio of 1:3 positive to negative predictions compared to the 1:2 ratio observed in the outcome variable
- `gender`, `length_of_stay`, `medication_type_c`, `diagnosis_code_D003`, `age` and `num_previous_admissions` were identified as important features in the SHAP plot

## Modelling engineered features only (non-text)

### Create dataframe for modelling

In [ ]:
#create df that uses engineered features that try and capture predictive signal from important features
e_df = df[engineered_features+['gender', 'readmitted_30_days']].copy()

#show df
e_df.head(2)

### Random Forest

#### Preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#Columns to encode
string_cols = ['gender']

#Instantiate OneHotEncoder
ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

#Fit and transform
encoded_array = ohe.fit_transform(e_df[string_cols])

#create a DataFrame from the encoded array
encoded_df = pd.DataFrame(
    encoded_array,
    columns=ohe.get_feature_names_out(string_cols),
    index=e_df.index
)

#drop original string columns and concatenate encoded columns
ranf_e_df = pd.concat([
    e_df.drop(columns=string_cols),
    encoded_df
], axis=1)

#show df
ranf_e_df.head(2)

#### Dataset creation

In [ ]:
#create X and y dataframe slices
X = ranf_e_df.drop(columns='readmitted_30_days').copy()
y = ranf_e_df[['readmitted_30_days']].copy()

#create stratified train-test split to handle class imbalance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

#show shapes of dataframes
print("\nDataset Shapes:")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

#show class distribution in y_train and y_test
print("\nClass distributions:")
print(y_train['readmitted_30_days'].value_counts(normalize=True))
print(y_test['readmitted_30_days'].value_counts(normalize=True))

#### Model training

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#instantiate Random Forest model using out-of-box hyperparameters
rf_model = RandomForestClassifier(
    n_estimators=100,               
    criterion='gini',               
    max_depth=None,                
    min_samples_split=2,          
    min_samples_leaf=1,            
    min_weight_fraction_leaf=0.0,  
    max_features='sqrt',          
    max_leaf_nodes=None,           
    min_impurity_decrease=0.0,     
    bootstrap=True,                
    oob_score=False,               
    n_jobs=None,                   
    random_state=42,               
    verbose=0,                     
    warm_start=False,              
    class_weight=None,             
    ccp_alpha=0.0,                 
    max_samples=None        
)

# Fit the model
rf_model.fit(X_train, y_train.values.ravel())  # ravel to avoid warning for 2D y

#### Performance

In [ ]:
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, f1_score, fbeta_score, roc_auc_score, roc_curve
)

import matplotlib.pyplot as plt

%matplotlib inline

#predict class labels
y_pred = rf_model.predict(X_test)

#predict class probabilities for ROC AUC
y_probs = rf_model.predict_proba(X_test)[:, 1]

#accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

#F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.3f}")

#F2 Score
f2 = fbeta_score(y_test, y_pred, beta=2)
print(f"F2 Score: {f2:.3f}")

#ROC AUC Score
roc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {roc:.3f}")

#confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf_model.classes_)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

### Observations

- Random Forest model trained on engineered features achieved marginally higher accuracy, F1 and F2 scores but had a small dip in ROC-AUC score performance compared to the equivalent model trained on non-engineered features

## Summary

### Data cleansing

Data was clean and well structured:
- Primary key was unique
- No missing values
- No mixed data formats in a single feature

No action was taken to cleanse the data.

### Exploratory Data Analysis

> **NOTE:** The feature `discharge_note` was omitted from modelling in this section.

#### Data inspection

- The dataset was fairly small, containing 200 rows.  
- It included both categorical features and features that could be interpreted as **discrete numeric** (e.g., `age`, `length_of_stay`, `num_previous_admissions`).  

#### Individual distributions

- The outcome variable `readmitted_30_days` displayed a moderate class imbalance of approximately 1:2 between the positive and negative classes.  
- Most features had a relatively small range of unique values. `age` had the most with 67 unique values, while all other features had ≤ 14 unique values.  
- Discrete features were generally well balanced, showing only mild class imbalance.  
- Discrete numeric features exhibited various distribution shapes — from **right-skewed** to **multimodal**.  

#### Distribution across the outcome variable

- No feature showed a strong or moderate correlation with the outcome variable.  
- `age` and `num_previous_admissions` showed mild correlation with `readmitted_30_days`.  
- Visualising feature distributions across the positive and negative classes of the outcome variable suggested that **particular value ranges** within some features might carry predictive signal (see relevant observations).  
- New features were derived based on these insights to enhance the model’s ability to capture this signal.  


### Modelling

#### Benchmark modelling

- Stratified sampling was used to handle the moderate class imbalance observed in the outcome variable, ensuring even representation of both classes across the train and test datasets.  
- A test-train split of 0.2 was chosen to provide enough datapoints in the training dataset for the machine learning algorithm to learn any trends, whilst leaving sufficient datapoints in the test set to allow generalisable inferences.  
- Random Forest was chosen as the benchmark algorithm due to its ability to model non-linear relationships and achieve reasonable baseline performance with minimal tuning.  
- The performance of the benchmark model was poor: marginally better than random choice, but worse than the baseline accuracy.  
- The SHAP plot identified `age`, `num_previous_admissions`, and `length_of_stay` as the three most important features — consistent with findings from the EDA when comparing these variables across outcome classes.  

#### Engineered features only model

- The predictive signal of the engineered features was modelled and compared to the benchmark model to evaluate whether a simplified representation of the dataset retained comparable predictive capability, and to derive insights into how specific values related to the outcome classes.  
- The same preprocessing, test-train split, and model configuration were used between the benchmark and engineered features only models to ensure a meaningful performance comparison.  
- The engineered features only model achieved weak overall performance, but similar to that of the benchmark model. This suggests that the limited predictive signal present in the raw features was largely captured in the engineered representation of the dataset.  

### Follow-up steps

#### Enrich dataset

- Due to the poor performance of both models on the dataset, I would seek to enrich the dataset to reach a baseline level of performance before considering alternative models or hyperparameter optimisation.  
- The dataset could be enriched **vertically** (more rows) and **horizontally** (more features).  
- 200 datapoints, whilst above the minimum recommended 10 datapoints per feature, is still a fairly small number to both train and test a model effectively, especially given the dimensionality of the dataframe.  
- Increasing the number of datapoints could improve generalisability, reduce overfitting, and allow more robust performance evaluation via cross-validation.  
- Enriching the dataset with additional features could also improve the predictive potential of the model. For instance, using features extracted from unstructured data (e.g., discharge notes via NLP pipelines) could enhance signal detection.  
- Synthetic upsampling of the underrepresented class in the outcome variable using techniques such as SMOTE could allow the model to learn class boundaries more effectively, improving recall and F1-score for the minority class.

#### Model comparison and optimisation
- After a baseline level of performance is achieved, comparison against different ML algorithms such as Logistic Regression, Feedforward Neural Networks, and Support Vector Classifiers could provide insight into the best-fitting model architecture for this problem.  
- Similarly, hyperparameter optimisation could further refine performance by tuning regularisation strength, learning rates, tree depths, or class weights depending on the model used.  
